In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import roc_auc_score,log_loss
from tqdm import tqdm
from sklearn.preprocessing import  OneHotEncoder,StandardScaler
import pandas_profiling
import datetime as dt
import lightgbm as lgbm
from random import normalvariate
from catboost import CatBoostClassifier
%pylab inline
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [84]:
df = pd.read_csv('data/uplift_train.csv')
df1 = pd.read_csv('data/clients.csv', parse_dates=['first_issue_date','first_redeem_date'])
df2 = pd.read_csv('data/products.csv')
test = pd.read_csv('data/uplift_test.csv')

In [85]:
df1['first_redeem_date']=df1['first_redeem_date'].fillna(dt.datetime(2019, 3, 19, 0, 0))
df1['first_issue_date'] = df1['first_issue_date'].apply(lambda x: x.timestamp())
df1['first_redeem_date'] = df1['first_redeem_date'].apply(lambda x: x.timestamp())
redeem_dates=df1[['client_id','first_redeem_date']].values

In [4]:
df1['redeem_diff'] = dt.datetime(2019, 3, 19, 0, 0).timestamp()-df1['first_redeem_date']
df1['diff'] = df1['first_redeem_date'] - df1['first_issue_date']

In [5]:
df1['age2'] = df1['age']
age_index = (df1['age'] < -900) & (df1['age'] > -1000)
df1.loc[age_index, 'age2'] = df1.loc[age_index, 'age'] + 1019

age_index = (df1['age'] > 900) & (df1['age'] < 1000)
df1.loc[age_index, 'age2'] = 1019 - df1.loc[age_index, 'age']

age_index = (df1['age'] > 1900) & (df1['age'] < 2000)
df1.loc[age_index, 'age2'] = 2019 - df1.loc[age_index, 'age']

age_index = (df1['age'] > 120) & (df1['age'] < 200)
df1.loc[age_index, 'age2'] = df1.loc[age_index, 'age'] - 100

age_index = (df1['age'] > 1800) & (df1['age'] < 1900)
df1.loc[age_index, 'age2'] = 1919 - df1.loc[age_index, 'age']

df1['age'] = df1['age2']
df1.drop('age2', axis=1, inplace=True)

df1['fake_age'] = df1['age'].apply(lambda x:int((x>90)or(x<16)))

In [6]:
data_train = pd.merge(df1,df)
data_test = pd.merge(df1,test)

In [7]:
#data_train['first_issue_date']=pd.to_datetime(data_train['first_issue_date'])
#data_train['first_redeem_date']=pd.to_datetime(data_train['first_redeem_date'])
#data_test['first_issue_date']=pd.to_datetime(data_test['first_issue_date'])
#data_test['first_redeem_date']=pd.to_datetime(data_test['first_redeem_date'])

In [8]:
#data_train = pd.merge(df1,df)
#miss = data_train[data_train['first_redeem_date'].isna()]
#miss['first_redeem_date']=miss['first_issue_date']
#data_train.first_redeem_date[miss.index]=miss.first_redeem_date.values
#data_train

In [9]:
#data_test=pd.merge(df1,test)
#miss = data_test[data_test['first_redeem_date'].isna()]
#miss['first_redeem_date']=miss['first_issue_date']
#data_test.first_redeem_date[miss.index]=miss.first_redeem_date.values
#data_test

In [11]:
#data_train['fake_age'] = data_train['age'].apply(lambda x:int((x>90)or(x<16)))
#data_test['fake_age'] = data_test['age'].apply(lambda x:int((x>90)or(x<16)))

In [12]:
#data_train['first_issue_date'] = data_train['first_issue_date'].apply(lambda x: x.timestamp())
#data_train['first_redeem_date'] = data_train['first_redeem_date'].apply(lambda x: x.timestamp())
#data_test['first_issue_date'] = data_test['first_issue_date'].apply(lambda x: x.timestamp())
#data_test['first_redeem_date'] = data_test['first_redeem_date'].apply(lambda x: x.timestamp())

In [13]:
purch = pd.read_csv('data/purchases.csv',parse_dates=['transaction_datetime'])

In [14]:
user_id = purch.drop_duplicates('client_id').client_id.values

In [15]:
transactions = purch.drop_duplicates('transaction_id')

In [94]:
transactions 

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,1.543648e+09,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
19,000012768d,c1ca85d462,1.544951e+09,5.7,0.0,0.0,0.0,574.0,ed8d2683fd,4dcf79043e,4.0,164.0,NaN
30,000012768d,6a0e96d0bc,1.552040e+09,8.0,0.0,0.0,0.0,803.0,017c89b915,36294f82c2,1.0,50.0,NaN
46,000012768d,b34f23306e,1.552576e+09,2.0,0.0,0.0,0.0,419.0,017c89b915,6bab126f22,1.0,59.0,NaN
52,000036f903,12b218b054,1.543402e+09,1.2,0.0,0.0,0.0,241.0,6381a55c22,f6ad31e071,0.0,32.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45786546,fffff6ce77,eba2694c0c,1.551359e+09,1.8,0.0,0.0,0.0,446.0,5ec64bdccb,4009f09b04,1.0,5.0,NaN
45786555,fffff6ce77,bdd578a115,1.551798e+09,0.3,0.0,0.0,0.0,247.0,5ec64bdccb,688e870c29,1.0,247.0,NaN
45786556,fffff6ce77,6d9fc7ad6b,1.551890e+09,0.1,0.0,0.0,0.0,109.0,5ec64bdccb,e0af1fd8d0,1.0,10.0,NaN
45786558,fffff6ce77,b8b6e5e851,1.551891e+09,0.2,0.0,0.0,0.0,245.0,6e4faad02e,0facc2a965,1.0,245.0,NaN


In [16]:
transactions['transaction_datetime'] = transactions['transaction_datetime'].apply(lambda x: x.timestamp())

In [96]:
redeem={i[0]:i[1] for i in redeem_dates}
transactions['redeem_date']=transactions['client_id'].apply(lambda x:redeem[x])

In [75]:
#число покупок
a = transactions[['client_id','transaction_id']].values
pok=[]
user=''
count=0
for row in a:
    if row[0]!=user:
        pok.append(count)
        user =row[0]
        count=1
    else:
        count+=1
pokupki = pok[1:]
pokupki.append(count)

In [113]:
x

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red,redeem_date,before_redeem,before_redeem_sum
0,000012768d,7e3e2e3984,1.543648e+09,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN,1.515094e+09,0,0.0
19,000012768d,c1ca85d462,1.544951e+09,5.7,0.0,0.0,0.0,574.0,ed8d2683fd,4dcf79043e,4.0,164.0,NaN,1.515094e+09,0,0.0
30,000012768d,6a0e96d0bc,1.552040e+09,8.0,0.0,0.0,0.0,803.0,017c89b915,36294f82c2,1.0,50.0,NaN,1.515094e+09,0,0.0
46,000012768d,b34f23306e,1.552576e+09,2.0,0.0,0.0,0.0,419.0,017c89b915,6bab126f22,1.0,59.0,NaN,1.515094e+09,0,0.0
52,000036f903,12b218b054,1.543402e+09,1.2,0.0,0.0,0.0,241.0,6381a55c22,f6ad31e071,0.0,32.0,NaN,1.492951e+09,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45786546,fffff6ce77,eba2694c0c,1.551359e+09,1.8,0.0,0.0,0.0,446.0,5ec64bdccb,4009f09b04,1.0,5.0,NaN,1.503766e+09,0,0.0
45786555,fffff6ce77,bdd578a115,1.551798e+09,0.3,0.0,0.0,0.0,247.0,5ec64bdccb,688e870c29,1.0,247.0,NaN,1.503766e+09,0,0.0
45786556,fffff6ce77,6d9fc7ad6b,1.551890e+09,0.1,0.0,0.0,0.0,109.0,5ec64bdccb,e0af1fd8d0,1.0,10.0,NaN,1.503766e+09,0,0.0
45786558,fffff6ce77,b8b6e5e851,1.551891e+09,0.2,0.0,0.0,0.0,245.0,6e4faad02e,0facc2a965,1.0,245.0,NaN,1.503766e+09,0,0.0


In [117]:
#число и сумма покупок до redeem date
x = transactions.copy()
x['before_redeem']=x['transaction_datetime']<x['redeem_date']
x['before_redeem']=x['before_redeem'].apply(lambda x:int(x))
x['before_redeem_sum']=x['before_redeem']*x['purchase_sum']
x=x[['client_id','before_redeem','before_redeem_sum']].values

array([['000012768d', 0, 0.0],
       ['000012768d', 0, 0.0],
       ['000012768d', 0, 0.0],
       ...,
       ['fffff6ce77', 0, 0.0],
       ['fffff6ce77', 0, 0.0],
       ['fffff6ce77', 0, 0.0]], dtype=object)

In [118]:
br_count=[]
br_sum=[]
user=''
count_br=0
sum_br=0

for row in x:
    if row[0]!=user:
        br_count.append(count_br)
        br_sum.append(sum_br)
        user =row[0]
        count_br=row[1]
        sum_br=row[2]
    else:
        count_br+=row[1]
        sum_br+=row[2]
br_count = br_count[1:]
br_sum = br_sum[1:]
br_count.append(count_br)
br_sum.append(sum_br)

In [18]:
#даты последних покупок
dates = purch.drop_duplicates('client_id',keep='last').transaction_datetime.values

In [19]:
#число магазинов
b = transactions[['client_id','store_id']].values
store=[]
user=''
count=0
for row in b:
    if row[0]!=user:
        store.append(count)
        user =row[0]
        count=1
        last_store = row[1]
    else:
        if row[1]!=last_store:
            count+=1
            last_store = row[1]
stores = store[1:]
stores.append(count)

In [20]:
#cредняя сумма покупки
c = transactions[['client_id','purchase_sum']].values
avsum=[]
user=''
count=1
sred=0
for row in c:
    if row[0]!=user:
        avsum.append(sred/count)
        user =row[0]
        count=1
        sred=row[1]
    else:
        count+=1
        sred+=row[1]
average = avsum[1:]
average.append(sred/count)

In [21]:
produkti = purch.product_id.values
useri = purch.client_id.values

In [22]:
alcohol = set(df2[df2['is_alcohol']==1].product_id.values)
owner = set(df2[df2['is_own_trademark']==1].product_id.values)

In [23]:
alc = lambda x: int(x in alcohol)
is_alc = [alc(i) for i in produkti]
own = lambda x: int(x in owner)
is_own = [own(i) for i in produkti]

In [54]:
purch['product_is_alco']=is_alc
purch['alc_sum']=purch['product_is_alco']*purch['trn_sum_from_iss']
purch['product_is_alco']=purch['product_is_alco']*purch['product_quantity']
purch['product_is_own']=is_own
purch['own_sum']=purch['product_is_own']*purch['trn_sum_from_iss']
purch['product_is_own']=purch['product_is_own']*purch['product_quantity']

In [287]:
#число купленного алкоголя и собственной торговой марки
a = purch.alc_sum.values
b = purch.own_sum.values
alc_count=[]
own_count=[]
alc_sum=[]
own_sum=[]
user=''
count_alc=0
count_own=0
sum_alc=0
sum_own=0
for i in range(len(useri)):
    if useri[i]!=user:
        alc_count.append(count_alc)
        own_count.append(count_own)
        alc_sum.append(sum_alc)
        own_sum.append(sum_own)
        user =useri[i]
        count_alc=is_alc[i]
        count_own=is_own[i]
        sum_alc=a[i]
        sum_own=b[i]        
    else:
        count_alc+=is_alc[i]
        count_own+=is_own[i]
        sum_alc+=a[i]
        sum_own+=b[i] 
alc_count = alc_count[1:]
own_count = own_count[1:]
alc_sum = alc_sum[1:]
own_sum = own_sum[1:]
alc_count.append(count_alc)
own_count.append(count_own)
alc_sum.append(sum_alc)
own_sum.append(sum_own)

In [360]:
new_features = pd.DataFrame({'client_id':user_id,'transaction_count':pokupki,'av_sum':average,
                             'last_transaction':dates,'store_count':stores,
                             'alc_count':alc_count,'alc_sum':alc_sum,
                             'own_count':own_count,'own_sum':own_sum,
                            'trans_br':br_count,'sum_br':br_sum})

In [361]:
new_features

,client_id,transaction_count,av_sum,last_transaction,store_count,alc_count,alc_sum,own_count,own_sum,trans_br,sum_br
0,000012768d,4,700.750000,2019-03-14 15:01:47,3,0,0.0,4,275.0,0,0.00
1,000036f903,32,306.406250,2019-03-17 10:29:37,14,1,229.0,14,1051.0,0,0.00
2,000048b7a6,8,471.500000,2019-03-12 10:28:19,3,0,0.0,3,393.0,8,3772.00
3,000073194a,17,564.788235,2019-03-16 09:41:21,1,9,3253.0,3,63.0,0,0.00
4,00007c7133,11,610.894545,2019-03-04 16:23:30,5,0,0.0,10,350.0,7,4159.00
...,...,...,...,...,...,...,...,...,...,...,...
400157,fffece623e,24,272.083750,2019-03-13 15:44:39,3,1,180.0,19,599.0,7,2634.58
400158,ffff3dfff8,17,427.343529,2019-03-17 16:50:01,1,2,440.0,13,1163.0,6,3458.45
400159,ffffaab9da,7,541.571429,2019-03-09 18:32:50,1,0,0.0,7,208.0,0,0.00
400160,ffffeb5619,7,2175.174286,2019-03-08 12:32:49,3,8,3903.0,10,464.0,7,15226.22


In [362]:
new_features['last_transaction'] = new_features['last_transaction'].apply(lambda x: x.timestamp())

In [350]:
new_features['transaction_sum'] = new_features['av_sum']*new_features['transaction_count']


In [ ]:
new_features['trans_ar']=new_features['transaction_count']-new_features['trans_br']
new_features['sum_ar']=new_features['transaction_sum']-new_features['sum_br']

In [292]:
new_features['alc_ratio']=new_features['alc_sum']/new_features['transaction_sum']
new_features['own_ratio']=new_features['own_sum']/new_features['transaction_sum']

In [383]:
new_features['store_ratio'] = new_features['store_count']/new_features['transaction_count']

In [384]:
data_train_full = pd.merge(data_train,new_features)
data_test_full = pd.merge(data_test,new_features)

In [385]:
data_train_full.index=data_train_full['client_id']
del data_train_full['client_id']

In [386]:
data_test_full.index=data_test_full['client_id']
del data_test_full['client_id']

In [387]:
X = data_train_full.drop(['target'],axis='columns')

In [388]:
Cat_cols = ['gender','fake_age']
Cat_cols1 = ['gender']
#Num_cols = ['first_issue_date', 'first_redeem_date', 'age','transaction_count', 'av_sum',
 #      'last_transaction', 'store_count', 'alc_count', 'own_count','transaction_sum','redeem_diff','diff']

In [389]:
Num_cols = list(X.columns.values)
Num_cols.remove('gender')
Num_cols.remove('fake_age')
Num_cols.remove('treatment_flg')
Num_cols

['first_issue_date',
 'first_redeem_date',
 'age',
 'redeem_diff',
 'diff',
 'transaction_count',
 'av_sum',
 'last_transaction',
 'store_count',
 'alc_count',
 'alc_sum',
 'own_count',
 'own_sum',
 'trans_br',
 'sum_br',
 'store_ratio']

In [393]:
Num_cols = ['first_issue_date',
 'first_redeem_date',
 'age',
 'redeem_diff',
 'diff',
 'transaction_count',
 'av_sum',
 'last_transaction',
 'store_count',
 'alc_count',
 'alc_sum',
 'own_count',
 'own_sum',
 'trans_br',
 'sum_br',
 ]

In [394]:
enc = OneHotEncoder(sparse=False)
scaler = StandardScaler()
X_cat_encoded = pd.DataFrame(enc.fit_transform(X[Cat_cols]))
X_num_scaled = pd.DataFrame(scaler.fit_transform(X[Num_cols]),columns=Num_cols)
Test_cat = pd.DataFrame(enc.fit_transform(data_test_full[Cat_cols]))
Test_num =  pd.DataFrame(scaler.fit_transform(data_test_full[Num_cols]),columns=Num_cols)
test1 = pd.concat([Test_cat,Test_num],axis=1)
train =pd.concat([X_cat_encoded,X_num_scaled,data_train['treatment_flg']],axis=1)
train_full = pd.concat([X_cat_encoded,X_num_scaled,data_train[['treatment_flg','target']]],axis=1)

In [165]:
cat_params = { 'task_type':'CPU',
              'loss_function':'Logloss', 'eval_metric':'Logloss',
               'iterations':20000, 'od_type': "Iter", 'od_wait':200
}

lgbm_params = {'learning_rate':0.01,'max_depth':6,'num_leaves':20, 'min_data_in_leaf':3, 
               'subsample':0.8, 'colsample_bytree': 0.8, 'reg_alpha':0.01,'max_bin':416,
               'bagging_freq':3,'reg_lambda':0.01,'num_leaves':20, 'n_estimators':600, 
               'eval_metric':'Logloss', 'application':'binary', 
               'iterations':20000, 'od_type': 'Iter', 'od_wait':200
}
grid = {'learning_rate': [0.01, 0.05,0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

In [166]:
def uplift_at_k(y_true, uplift, treatment, k=0.3):
    order = np.argsort(-uplift)
    treatment_n = int((treatment == 1).sum() * k)
    treatment_p = y_true[order][treatment[order] == 1][:treatment_n].mean()
    control_n = int((treatment == 0).sum() * k)
    control_p = y_true[order][treatment[order] == 0][:control_n].mean()
    score_at_k = treatment_p - control_p
    
    return score_at_k

In [169]:
def trans_train_model(model, df_X, df_X_test, num_folds=5, random_state=0, verbose=2):

    cat_params['random_state'] = random_state
    
    df_X['new_target'] = (df_X['target'] + df_X['treatment_flg'] + 1) % 2
    df_y = df_X[['new_target']]
    treatment = df_X['treatment_flg'].to_numpy()
    old_target = df_X['target'].to_numpy()
    df_X = df_X.drop(['target', 'new_target', 'treatment_flg'], axis=1, errors='ignore')    
    
    X = df_X.to_numpy()
    y = df_y.to_numpy()
    X_test = df_X_test.to_numpy()
    
    folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=random_state)
    scores = []
    uplift_scores = []    
    prediction = np.zeros(len(X_test))

    for i, (train_index, valid_index) in enumerate(folds.split(X, y)):    
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]        
        treat_valid = treatment[valid_index]
        old_target_vaild = old_target[valid_index]
        
        if (model == 'catboost'):
            f = CatBoostClassifier(**cat_params)
            f.fit(X_train, y_train, eval_set=(X_valid, y_valid), use_best_model=True, verbose=False)
        elif (model == 'lgbm'):
            f = lgbm.LGBMClassifier(**lgbm_params)        
            f.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False)
    
        y_pred_valid = f.predict_proba(X_valid)[:, 1]
        score = log_loss(y_valid, y_pred_valid)
        uplift_score = uplift_at_k(old_target_vaild, y_pred_valid, treat_valid)
        uplift_scores.append(uplift_score)
        if (verbose > 1):
            print('OOF Uplift score: {0:.5f}'.format(uplift_score))
        
        scores.append(score)
        # predict on test and accumulate the result
        y_pred = f.predict_proba(X_test)[:, 1]
        prediction += y_pred

    # get average prediction from all models
    prediction /= num_folds
  
    if (verbose > 0):
        print('CV mean score: {0:.5f}, std: {1:.5f}'.format(np.mean(scores), np.std(scores)))    
        print('Uplift score @30%: {0:.5f}, std: {1:.5f}'.format(np.mean(uplift_scores), np.std(uplift_scores)))    
        
    return prediction

In [53]:
pred = trans_train_model(train_full, test,verbose=2)

OOF Uplift score: 0.08902
OOF Uplift score: 0.07504
OOF Uplift score: 0.07946
OOF Uplift score: 0.07788
OOF Uplift score: 0.07632
CV mean score: 0.68868, std: 0.00017
Uplift score @30%: 0.07954, std: 0.00496


In [49]:
pred = trans_train_model(train_full, test, random_state=17,verbose=2)

OOF Uplift score: 0.08529
OOF Uplift score: 0.09015
OOF Uplift score: 0.06917
OOF Uplift score: 0.08345
OOF Uplift score: 0.07923
CV mean score: 0.68867, std: 0.00043
Uplift score @30%: 0.08146, std: 0.00707


In [219]:
pred = trans_train_model('catboost',train_full, test1, random_state=29,verbose=2)

OOF Uplift score: 0.08291
OOF Uplift score: 0.08300
OOF Uplift score: 0.08787
OOF Uplift score: 0.07961
OOF Uplift score: 0.08030
CV mean score: 0.68848, std: 0.00017
Uplift score @30%: 0.08274, std: 0.00290


In [395]:
pred37 = trans_train_model('catboost',train_full, test1, random_state=37,verbose=2)

OOF Uplift score: 0.07730
OOF Uplift score: 0.08600
OOF Uplift score: 0.08031
OOF Uplift score: 0.08920
OOF Uplift score: 0.08637
CV mean score: 0.68865, std: 0.00012
Uplift score @30%: 0.08384, std: 0.00436


In [396]:
df_submission = pd.DataFrame({'client_id':data_test_full.index,'uplift': pred37})
df_submission.to_csv('sub_37.csv', index=False)

In [57]:
data_test_full.index

Index(['000048b7a6', '000073194a', '00007c7133', '00007f9014', '0000a90cf7',
       '0000b59cec', '0000bb4e4e', '0000bcec9c', '0000eecb82', '0000f0ecdb',
       ...
       'fffb310977', 'fffbfc9346', 'fffc68800d', 'fffc6fd3ea', 'fffde49bc6',
       'fffeacbe3e', 'fffece623e', 'ffff3dfff8', 'ffffaab9da', 'ffffeb5619'],
      dtype='object', name='client_id', length=200123)

In [137]:
sol1.index = sol1['client_id']
del sol1['client_id']

In [138]:
sol1

,uplift
client_id,
000048b7a6,0.056886
000073194a,0.037388
00007c7133,0.038690
00007f9014,0.017279
0000a90cf7,0.015631
...,...
fffeacbe3e,0.033306
fffece623e,0.035032
ffff3dfff8,0.052328


In [139]:
sol1.to_csv('sol2.csv')

In [113]:
from catboost import CatBoostClassifier

In [133]:
cbc = CatBoostClassifier(thread_count=2, random_state=17, silent=True)
cbc.fit(train,data_train['target'])

In [134]:
preds1 = [i[1] for i in cbc.predict_proba(test1)]
preds0 = [i[1] for i in cbc.predict_proba(test0)]